In [ ]:
import pyspark.sql.functions as f
from pyspark.sql import Window

# Window Functions

In this notebook we'll introduce window functions, which are a powerful tool for working with data in SQL.

Window functions in PySpark (and in SQL in general) are a tool for performing complex analytical tasks within PySpark DataFrames. They allow you to perform **calculations across a group of rows related to the current row, without aggregating the entire dataset**. This makes them very useful for tasks like calculating moving averages, ranking items within groups, and calculating cumulative sums.

**Window functions allow you to calculate aggregate functions (like sum, average, max, min) over a specific window of rows**.

Also, using window functions can be more efficient than traditional group by aggregations, especially when you want to include aggregated values alongside individual rows without reducing the number of rows.

In this section, we're going to explore the main PySpark window functions. Let's start by downloading a dummy dataset of employees and load it into a DataFrame.

In [ ]:
%sh

wget https://raw.githubusercontent.com/inesmcm26/lp-big-data/main/data/employees.csv

In [ ]:
df_employees = (
    spark.read.format('csv')
    .option('inferSchema', 'true')
    .option('header', 'true')
    .option('sep', ',')
    .load('file:/databricks/driver/employees.csv')
)

df_employees.display()

To use Window functions, we first need to define a `Window`. This class is part of the PySpark SQL library and allows you to perform calculations across a group of rows related to one another. [Here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Window.html) you can find the official documentation of this class.

It has three main methods:

- `partitionBy(col/cols)`: Partitioning is the process of dividing the dataset into groups of rows based on some criteria. This method is used to define these partitions.
- `orderBy(col/cols)`: This method defines the order in which rows within each partition are processed. If no partition is defined, the whole dataset is considered.
- `rowsBetween(start, end)`: Defines the range of rows included in a window frame for window functions. For each row, the window function will only consider the rows within the specified window range.

## Aggregation functions

We've seen some aggregation function in the previous module. These were used to aggregate the values resulting from a `groupBy` operation.

However, these functions can also be used with Window operations.

Let's answer some questions to exemplify.

1. For each employee, what is the average salary in their department?

In [ ]:
# First we define our Window object.
# We want to partition by 'department'
window = Window.partitionBy('department')

# Next we define an aggregation function over the window
avg_salary_fn = f.avg(f.col('salary')).over(window)

# To get the result we need to apply the window function to a dataframe
# To do that, we create a new column with the result of the window function
df_employees.withColumn('avg_salary_department', avg_salary_fn).display()

2. For each employee, what is the maximum salary in their department?

We have two ways of answering this question

In [ ]:
# 1. Partition by department
window = Window.partitionBy('department')

# Define the `max` aggregation function
max_salary_fn = f.max('salary').over(window)

# Apply the function to the dataframe
df_employees.withColumn('max_salary_department', max_salary_fn).display()

In [ ]:
# 2. Partition by department and order the salary values in descending
# order within each Window frame
window = Window.partitionBy('department').orderBy(f.desc('salary'))

# Define the `first` aggregation function to get the first salary
# in each window frame
max_salary_fn = f.first('salary').over(window)

# Apply the function to the dataframe
df_employees.withColumn('max_salary_department', max_salary_fn).display()

3. For each employee, what is the average salary per gender in their department?

We can also partition the dataset using more than 1 column

In [ ]:
# First we define our Window object.
# We want to partition by 'department' and 'gender'
window = Window.partitionBy(['department', 'gender'])

# Next we apply an aggregation fucntion over the window
avg_salary_fn = f.avg(f.col('salary')).over(window)

# To get the result from the window function we need to apply it to a dataframe
df_employees.withColumn('avg_salary_sex_department', avg_salary_fn).display()

I know what you're thinking. The way we answered these questions is very similar to using a `groupBy` operation. In fact, in questions ***1*** and ***2***, for instance, all employees from the same department have the same aggregated value.

So, why use window functions? Mainly because the questions started with 'For each employee...', which makes it useful to have a new column with the aggregated value for each employee.

If, for example, question ***1*** was 'What is the average salary in each department?', then a `groupBy` operation would be enough.


As you can see, sometimes it might be useful to maintain the dataset shape as is. Window functions enable you to compute aggregated values alongside detailed information from other columns in the dataset. This allows for more complex analyses and reporting scenarios where you need both aggregated summaries and detailed information.

Let's see a more complex example:

4. Imagine you want to calculate the difference between each employee's salary and the average salary of the five employees closest in salary, including the employee itself (two with higher salaries and two with lower salaries)

Let's break this question down:
- We want to calculate the difference between each employee's salary and an average salary
- The average salary should be calculated considering only 5 employees' salaries
- These 5 employees are the two above in terms of salary order, the two below, and the employee himself

In [ ]:
# Define a window with a limited size: between the 2 before and 2 after rows
window = Window.orderBy(f.desc('salary')).rowsBetween(-2, 2)

(
    df_employees
    .withColumn(
        'avg_salary',
        # Apply the aggregation function to the limited window
        f.avg('salary').over(window)
    )
    .withColumn(
        'salary_diff',
        f.col('salary') - f.col('avg_salary')
    )
).display()

## Row function

Besides aggregation functions, we can also apply the `row_number()` function over a window.

This function assigns a unique sequential integer to each row within a partition, according to the order in which the rows appear in the partition. The first row gets the number 1, the second row gets the number 2, and so on.

Let's rank the employees by salary in descending order.

In [ ]:
# First, we create a window ordered by 'salary'
# Since this window is not partitioned, the whole dataset is used for sorting
window = Window.orderBy(f.desc('salary'))

# Apply the `row_number` window function to the dataset
# over the defined window
(
    df_employees
    .withColumn('row_number', f.row_number().over(window))
).display()

## Rank function

The `rank()` function is similar to the `row_number()` function, but it does not assign a unique sequential integer to each row. Instead, it assigns the same rank to rows with the same value.

Let's rank the employees by salary in descending order using the `rank()` function and compare the results with the `row_number()` function.

In [ ]:
# First, we create a window ordered by 'salary'
# Since this window is not partitioned, the whole dataset is used for sorting
window = Window.orderBy(f.desc('salary'))

# Apply the `rank` window function to the dataset
# over the defined window
(
    df_employees
    .withColumn('row_number', f.rank().over(window))
).display()

See the difference? Jessica and Ivy got the same rank because they have the same salary.

Now let's apply the `rank()` to find the salary ranking by gender.

This time, the rank will be calculated for each partition of the column `gender`.

In [ ]:
window = Window.partitionBy('gender').orderBy(f.desc('salary'))

(
    df_employees
    .withColumn('salary_rank', f.rank().over(window))
).display()

## Dense Rank Function

This function works similarly to the `rank()` function, but it does not leave gaps between consecutive ranks. If two rows have the same value, they will receive the same rank, and the next row will receive the next rank available.

In [ ]:
window = Window.partitionBy('gender').orderBy(f.desc('salary'))

(
    df_employees
    .withColumn('salary_rank', f.dense_rank().over(window))
).display()

## Lead and Lag Functions

Lead and lag are functions that are applied to each dataset row, within a window partition.

Lead returns the value of a column at a specified offset after the current row.
Lag returns the value of a column at a specified offset before the current row.

Let's look at the salary difference between employees within a department. We want to quantify the salary difference between each employee and the next employee earning more within the same department.

In [ ]:
# Define the window
window = Window.partitionBy('department').orderBy('salary')

(
    df_employees
    .withColumn('next_salary', f.lead(col=f.col('salary'), offset=1).over(window))
    .withColumn(
        'salary_diff',
        f.col('next_salary') - f.col('salary')
    )
).display()

We could also do the reverse operation and see how much more each employee earns compared to the next employee earning less within the same department.

In [ ]:
# Define the window
window = Window.partitionBy('department').orderBy('salary')

(
    df_employees
    .withColumn('prev_salary', f.lag(col=f.col('salary'), offset=1).over(window))
    .withColumn(
        'salary_diff',
        f.col('salary') - f.col('prev_salary')
    )
).display()

---

Now that you know how to use the main window functions, go to the `exercises` notebook and practice what you've learned!

## Exercises

Now that you know how to use the main window functions, let's get back to our orders dataset and practice!

In [ ]:
df_orders = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .option("sep", ",")
    .load("/FileStore/lp-big-data/preprocessed-data/orders-data/orders_preprocessed.csv")
)

df_products = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .option("sep", ",")
    .load("/FileStore/lp-big-data/preprocessed-data/orders-data/products_preprocessed.csv")
)

df_orders_products = (
    df_orders.join(
        df_products,
        on=['product_id'],
        how='left'
    )
)

df_orders_products.display()

1. Rank each customer based on the total amount of ordered products.

The ranking should have no gaps.

In [ ]:
# First, get the total amount of ordered products by customer
df_total_amount = (
    df_orders_products
    .groupBy('customer_id')
    .agg(f.sum('amount').alias('total_amount'))
)

df_total_amount.display()

In [ ]:
# Define the window ordered by 'total_amount' in descending order
window = Window.orderBy(f.desc('total_amount'))

(
    df_total_amount
    # Rank the customers by their total amount of ordered products
    .withColumn('customer_rank', f.dense_rank().over(window))
).display()

2. For each order, what is the difference in days between the current order's placing date and the last order's placing date for the same customer?

In [ ]:
# Define the window: partition by 'customer_id' and order by 'placing_date'
window = Window.partitionBy('customer_id').orderBy('placing_date')

(
    df_orders_products
    # Get the previous order date for each order
    .withColumn(
        'prev_order_date',
        f.lag('placing_date').over(window)
    )
    # Calculate the difference in days between the current and previous order
    .withColumn(
        'days_diff',
        f.datediff(
            f.col('placing_date'),
            f.col('prev_order_date')
        )
    )
    .select(
        'customer_id',
        'order_id',
        'placing_date',
        'prev_order_date',
        'days_diff'
    )
).display()

3. For each customer, what is the difference in revenue between each of his orders and the average revenue of the previous three placed orders?

In [ ]:
# Define the window: partition by 'customer_id', order by 'placing_date' and select the last 3 rows
window = Window.partitionBy('customer_id').orderBy('placing_date').rowsBetween(-3, 0)

(
    df_orders_products
    # Calculate the average revenue for the last 3 orders
    .withColumn(
        'avg_revenue',
        f.avg('revenue').over(window)
    )
    # Calculate the difference between the total price and the average revenue
    .withColumn(
        'revenue_diff',
        f.col('revenue') - f.col('avg_revenue')
    )
).display()

4. For each customer, what is the absolute difference in days between each of his order's delivery date and the next order's placing date, where the next order's placing date is before than the current order's delivery date?

Let's break the question down:
- We want to calculate the difference in days between each order's delivery date and the next order's placing date
- The next order's placing date should be before the current order's delivery date

In [ ]:
# Define the window: partition by 'customer_id', order by 'placing_date'
window = Window.partitionBy('customer_id').orderBy('placing_date')

(
    df_orders_products
    # Get the next order placing date for each order
    .withColumn(
        'next_placing_date',
        f.lead('placing_date').over(window)
    )
    # Calculate the difference in days between the current
    # order delivery date and next order placing date
    .withColumn(
        'days_diff',
        f.datediff(f.col('delivery_date'), f.col('next_placing_date'))
    )
    # Filter the rows where the next order was placed before the current order was delivered
    .filter(f.col('next_placing_date') < f.col('delivery_date'))
    .select(
        'customer_id',
        'placing_date',
        'delivery_date',
        'next_placing_date',
        'days_diff'
    )
).display()

5. What is the yearly average profit increase or decrease (difference between year Y and year Y-1) for each supplier?

In [ ]:
# First, get the average profit for each supplier in each year
df_avg_profit = (
    df_orders_products
    .groupBy(['order_year', 'supplier_id'])
    .agg(f.avg('profit').alias('avg_profit'))
)

df_avg_profit.display()

In [ ]:
# Next, define the window: partition by 'supplier_id', order by 'order_year'
window = Window.partitionBy('supplier_id').orderBy('order_year')

(
    df_avg_profit
    # Get the previous year average profit for each supplier
    .withColumn('prev_year_avg_profit', f.lag('avg_profit').over(window))
    # Calculate the difference between the average profit and the previous year average profit
    .withColumn('profit_diff', f.col('avg_profit') - f.col('prev_year_avg_profit'))
).display()

6. Determine the top 3 products with the highest profit margin.

Profit margin refers to the ratio between profit and revenue:

$profit\_margin_{product} = profit_{product} / revenue_{product}$

$profit_{product} = revenue_{product} - cost_{product}$

So let's break this question down:
- To calculate the cost of each product's order, we need to multiply the cost per unit by the quantity ordered
- Then, we want to get the total revenue and total cost of each product, for each supplier country
- Next, the profit is calculated as the difference between revenue and cost and the profit margin is calculated as the ratio between profit and revenue
- Finally, we rank the products by profit margin and select the top 3

In [ ]:

# First, calculate the total cost of each product order
df_cost = (
    df_orders_products
    .withColumn('cost', f.col('amount') * f.col('cost_per_unit'))
)

df_cost.display()

In [ ]:
# Then, get the total cost and revenue of each product, for each supplier country
df_grouped = (
    df_cost
    .groupBy(['supplier_country', 'product_id'])
    .agg(
        f.sum('cost').alias('total_cost'),
        f.sum('revenue').alias('total_revenue')
    )
)

df_grouped.display()

In [ ]:
# Next, calculate the profit and profit margin for each product
df_profit_margin = (   
    df_grouped
    # Calculate the profit
    .withColumn('profit', f.col('total_revenue') - f.col('total_cost'))
    # Calculate the profit margin
    .withColumn('profit_margin', f.col('profit') / f.col('total_revenue'))
)

df_profit_margin.display()

In [ ]:
# Finally, rank the products by profit margin

# Define the window: partition by 'supplier_country', order by 'profit_margin' in descending order
window = Window.partitionBy('supplier_country').orderBy(f.desc('profit_margin'))

(
    df_profit_margin
    # Rank the products by their profit margin
    .withColumn('rank', f.rank().over(window))
    # Filter the top 3 products by profit margin
    .filter(f.col('rank') <= 3)
).display()

Here is the whole solution:

In [ ]:
(
    df_orders_products
    # Calculate cost per order
    .withColumn('cost', f.col('amount') * f.col('cost_per_unit'))
    # Get total cost and revenue of each product, for each supplier country
    .groupBy(['supplier_country', 'product_id'])
    .agg(
        f.sum('cost').alias('total_cost'),
        f.sum('revenue').alias('total_revenue')
    )
    # Calculate the profit
    .withColumn('profit', f.col('total_revenue') - f.col('total_cost'))
    # Calculate the profit margin
    .withColumn('profit_margin', f.col('profit') / f.col('total_revenue'))
    # Rank the products by their profit margin
    .withColumn('rank', f.rank().over(window))
    # Filter the top 3 products by profit margin
    .filter(f.col('rank') <= 3)
).display()

7. **BONUS** Calculate the cumulative sum of average revenue generated by each customer over the years

Hint: Check all Window methods and attributes [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Window.html)

In [ ]:
window = (
    Window
    .partitionBy('customer_id')
    .orderBy('order_year')
    # Define the window frame: from the first year to the current year
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

(
    df_orders_products
    # Calculate the total revenue for each customer in each year
    .groupBy('customer_id', 'order_year')
    .agg(f.sum('revenue').alias('total_revenue'))
    # Calculate the cumulative revenue for each customer over the years
    .withColumn(
        'cum_revenue',
        f.sum('total_revenue').over(window)
    )
).display()